In [2]:
import os
import pandas as pd
from collections import Counter

# Example usage
folder_path = 'AT1 dataset_AusRadioTalkback'
data = []


for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            data.append({'filename': filename, 'content': content})


df = pd.DataFrame(data)
print(df)


                     filename  \
0   1_Dataset Description.txt   
1             ABCE1-plain.txt   
2             ABCE2-plain.txt   
3             ABCE3-plain.txt   
4             ABCE4-plain.txt   
5            ABCNE1-plain.txt   
6            ABCNE2-plain.txt   
7             COME1-plain.txt   
8             COME2-plain.txt   
9             COME3-plain.txt   
10            COME4-plain.txt   
11            COME5-plain.txt   
12            COME6-plain.txt   
13            COME7-plain.txt   
14            COME8-plain.txt   
15           COMNE1-plain.txt   
16           COMNE2-plain.txt   
17           COMNE3-plain.txt   
18           COMNE4-plain.txt   
19           COMNE5-plain.txt   
20           COMNE6-plain.txt   
21           COMNE7-plain.txt   
22             NAT1-plain.txt   
23             NAT2-plain.txt   
24             NAT3-plain.txt   
25             NAT4-plain.txt   
26             NAT5-plain.txt   
27             NAT6-plain.txt   
28             NAT7-plain.txt   
29        

In [3]:
#Modified code from Lab 2 Part 2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import string

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

class TextPreprocessor:
    def __init__(self, custom_punctuation=None, custom_stopwords=None):
        self.punctuation = string.punctuation
        if custom_punctuation:
            self.punctuation += custom_punctuation

        self.stop_words = set(stopwords.words('english'))
        if custom_stopwords:
            self.stop_words.update(custom_stopwords)

        self.stemmer = PorterStemmer()

    def remove_punctuation(self, text):
        return ''.join([char for char in text if char not in self.punctuation])

    def add_space_after_parenthesis(self, text):
        return re.sub(r'\)', ') ', text)

    def hyphen_to_space(self, text):
        return re.sub(r'\-', ' ', text)

    def to_lowercase(self, text):
        return text.lower()

    def remove_stopwords(self, text):
        words = word_tokenize(text)
        return ' '.join([word for word in words if word not in self.stop_words])
        
    def stem_words(self, text):
        words = word_tokenize(text)
        return ' '.join([self.stemmer.stem(word) for word in words])

    #Order matters - how you call these methods is how the text will be processed step-by-step
    def preprocess(self, text):
        text = self.add_space_after_parenthesis(text)
        text = self.hyphen_to_space(text)
        text = self.remove_punctuation(text)
        text = self.to_lowercase(text)
        text = self.remove_stopwords(text)
        text = self.stem_words(text)
        return text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mason\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
print(df)

                     filename  \
0   1_Dataset Description.txt   
1             ABCE1-plain.txt   
2             ABCE2-plain.txt   
3             ABCE3-plain.txt   
4             ABCE4-plain.txt   
5            ABCNE1-plain.txt   
6            ABCNE2-plain.txt   
7             COME1-plain.txt   
8             COME2-plain.txt   
9             COME3-plain.txt   
10            COME4-plain.txt   
11            COME5-plain.txt   
12            COME6-plain.txt   
13            COME7-plain.txt   
14            COME8-plain.txt   
15           COMNE1-plain.txt   
16           COMNE2-plain.txt   
17           COMNE3-plain.txt   
18           COMNE4-plain.txt   
19           COMNE5-plain.txt   
20           COMNE6-plain.txt   
21           COMNE7-plain.txt   
22             NAT1-plain.txt   
23             NAT2-plain.txt   
24             NAT3-plain.txt   
25             NAT4-plain.txt   
26             NAT5-plain.txt   
27             NAT6-plain.txt   
28             NAT7-plain.txt   
29        

In [5]:
for row, index in df.iterrows():
    #print(df.iloc[row]['content'])
    pass

    #overwrite the content of the row with a tokenised version I think
    #Lab 2 Part 2 DELETEE
#https://colab.research.google.com/drive/1yM9TayztqQw9Z7gzcS4fnUoj5ssDd84q?usp=sharing

In [6]:
#Remove first entry (the dataset description)
df = df.iloc[1:]

In [7]:
df

,filename,content
1,ABCE1-plain.txt,Thanks for that John Hall now John Hall will ...
2,ABCE2-plain.txt,Ah look l Les Pete.\n.\n Simon.\n G'day Peto....
3,ABCE3-plain.txt,If you haven't been with us before this how i...
4,ABCE4-plain.txt,Uh blue-tongues'd be unlikely to eat them be...
5,ABCNE1-plain.txt,A very good afternoon to you Roly.\n Good aft...
6,ABCNE2-plain.txt,And Greg Kerrin is my guest. Hello Greg.\n G'...
7,COME1-plain.txt,Good morning and welcome to another Two G B w...
8,COME2-plain.txt,Good morning everyone and welcome to a very f...
9,COME3-plain.txt,The doctor is in the lines are open one-three...
10,COME4-plain.txt,Morning Mark.\n\n Uh uh good morning John. Um...


In [30]:
#Code that combines all the different df rows based on filename into 1 long string grouped by radio Station

#Create prefix column using regex (take characters from beginning of filename, until you hit a number)
df['prefix'] = df['filename'].apply(lambda x: re.match(r'^[A-Z]+', x).group())

#Group by prefix
df_combined = df.groupby('prefix')['content'].apply(' '.join).reset_index()

In [40]:
df_combined

,prefix,content
0,ABCE,Thanks for that John Hall now John Hall will ...
1,ABCNE,A very good afternoon to you Roly.\n Good aft...
2,COME,Good morning and welcome to another Two G B w...
3,COMNE,Freo Dockers skipper there Peter Bell hello e...
4,NAT,One-eight-hundred-eight-oh-two-three-four-one...


In [50]:
# Custom punctuation and stopwords
custom_punctuation = "''""…""...""``" '"' '-'  # Add any custom punctuation marks here
custom_stopwords = []  # Add any custom stopwords here

# Create preprocessor
preprocessor = TextPreprocessor(custom_punctuation, custom_stopwords)

# Preprocess the text
#preprocessed_text = preprocessor.preprocess(article_text)

preprocessed = df_combined.iloc[0]['content']
# Preprocess the text
processed = preprocessor.preprocess(preprocessed)

In [52]:
# Get word frequencies
def get_word_frequencies(text, top_n=10):
    words = word_tokenize(text)
    word_freq = Counter(words)
    return word_freq.most_common(top_n)

word_freq = get_word_frequencies(processed, top_n=20)

print("\nTop 20 most frequent words:")
for word, count in word_freq:
    print(f"{word}: {count}")


Top 20 most frequent words:
uh: 580
um: 286
yeah: 232
well: 200
like: 167
ye: 166
that: 146
get: 139
oh: 133
think: 130
one: 123
good: 118
got: 111
go: 109
would: 99
realli: 95
sort: 94
look: 85
thank: 84
theyr: 82


In [62]:
# Custom punctuation and stopwords
custom_punctuation = "''""…""...""``" '"' '-'  # Add any custom punctuation marks here
custom_stopwords = ["uh", "well", "um", "yes", "yeah", "theyre", "get", "that", "like", "oh", "thank", "ive", "yep", "c", "okay", "your", "you're"]  # Add any custom stopwords here
#yes and theyre are the full non-stemmed versions of "ye" and 'theyr"

# Create preprocessor
preprocessor = TextPreprocessor(custom_punctuation, custom_stopwords)

# Preprocess the text
#preprocessed_text = preprocessor.preprocess(article_text)

preprocessed = df_combined.iloc[0]['content']
# Preprocess the text
processed = preprocessor.preprocess(preprocessed)

word_freq = get_word_frequencies(processed, top_n=50)

print("\nTop 20 most frequent words:")
for word, count in word_freq:
    print(f"{word}: {count}")


Top 20 most frequent words:
that: 146
think: 130
one: 123
good: 118
got: 111
go: 109
would: 99
realli: 95
sort: 94
look: 85
im: 77
know: 76
two: 74
dont: 73
ih: 72
say: 71
book: 70
want: 69
see: 66
bit: 66
actual: 66
mm: 62
id: 61
yknow: 59
thing: 59
right: 56
there: 56
use: 55
littl: 53
time: 53
plant: 51
put: 49
year: 49
call: 48
come: 48
thought: 48
your: 48
take: 47
youv: 47
read: 47
could: 46
much: 45
paint: 44
someth: 42
three: 41
love: 41
old: 40
lot: 39
quit: 39
also: 38
